In [1]:
from Tirto import Tirto #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [6]:
obj = Tirto()
end_date = date(2018, 8, 8)
start_date = date(2018, 1, 30)
init_page = 2

In [ ]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, single_date.strftime("%Y-%m-%d"))
    init_page = 1

2018-02-02
page  1
https://tirto.id/hasil-perempat-final-india-open-2018-empat-wakil-indonesia-lolos-cEdv
Insert berita  Hasil Perempat Final India Open 2018: Empat Wakil Indonesia Lolos
masuk
https://tirto.id/sandi-harap-ok-otrip-di-tanah-abang-bisa-diterapkan-bulan-depan-cEdu
Insert berita  Sandi Harap Ok-Otrip di Tanah Abang Bisa Diterapkan Bulan Depan
masuk
https://tirto.id/komisioner-komnas-ham-nilai-rancu-perluasan-pasal-penistaan-agama-cEds
Insert berita  Komisioner Komnas HAM Nilai Rancu Perluasan Pasal Penistaan Agama
masuk
https://tirto.id/hasil-india-open-2018-hendra-ahsan-ke-semifinal-cEdq
Insert berita  Hasil India Open 2018: Hendra/Ahsan ke Semifinal
masuk
https://tirto.id/dinkes-angka-gizi-buruk-di-jakarta-utara-turun-82-persen-cEdr
Insert berita  Dinkes: Angka Gizi Buruk di Jakarta Utara Turun 82 Persen
masuk
https://tirto.id/komisi-i-dpr-beda-pandangan-soal-mou-tni-polri-tangani-unjuk-rasa-cEdp
Insert berita  Komisi I DPR Beda Pandangan Soal MoU TNI-Polri Tangani Unjuk

masuk
https://tirto.id/jadwal-dan-siaran-langsung-liga-inggris-3--4-februari-2018-cEb4
Insert berita  Jadwal dan Siaran Langsung Liga Inggris 3 - 4 Februari 2018
masuk
https://tirto.id/presiden-jokowi-belum-terima-usulan-pansus-soal-dewan-pengawas-kpk-cEbZ
Insert berita  Presiden Jokowi Belum Terima Usulan Pansus Soal Dewan Pengawas KPK
masuk
https://tirto.id/demokrat-belum-pastikan-ahy-jadi-capres-2019-cEbU
Insert berita  Demokrat Belum Pastikan AHY Jadi Capres 2019
masuk
